In [1]:
!pip list

Package                       Version
----------------------------- ---------------------
absl-py                       1.0.0
adapter-transformers          3.0.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                0.1.12
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.0
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.0
attrs                         21.4.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.9.1
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.0
blis 

In [2]:
!pip install transformers
!pip install pytorch_lightnin
!pip install adapter-transformers
!pip install datasets
!pip install sentencepiece


     |████████████████████████████████| 4.0 MB 26.8 MB/s 
     |████████████████████████████████| 77 kB 6.9 MB/s 
     |████████████████████████████████| 596 kB 60.0 MB/s 
     |████████████████████████████████| 6.6 MB 51.9 MB/s 
     |████████████████████████████████| 895 kB 58.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: Could not find a version that satisfies the requirement pytorch_lightnin (from versions: none)
ERROR: No matching distribution found for pytorch_lightnin
     |████████████████████████████████| 3.9 MB 27.1 MB/s 
     |████████████████████████████████| 325 kB 17.6 MB/s 
     |████████████████████████████████| 212 kB 30.1 MB/s 
     |████████████████████████████████| 136 kB 48.3 MB/s 
     |████████████████████████████████| 1.1 MB 62.4 MB/s 
     |████████████████████████████████| 127 kB 64.1 MB/s 
     |████████████████████████████████| 144 kB 64.6 MB

In [2]:
!nvidia-smi

Thu Apr 28 03:23:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
base_path = "t5-small"
# base_path = "bert-base-cased"
adapter_path = "paraphrase"

In [2]:
import pandas as pd
from datasets import Dataset

In [3]:
from transformers import T5Tokenizer, BertTokenizer
tokenizer = T5Tokenizer.from_pretrained(base_path)
# tokenizer = BertTokenizer.from_pretrained(base_path)



In [10]:
!unzip COMBINED.zip

Archive:  COMBINED.zip
   creating: COMBINED/
  inflating: COMBINED/train.tsv      
  inflating: COMBINED/test.tsv       


In [4]:
df_test = pd.read_csv('COMBINED/test.tsv', sep = '\t', names=['in', 'expected'])
df_train = pd.read_csv('COMBINED/train.tsv', sep = '\t', names=['in', 'expected'])
df_test

,in,expected
0,"PCCW's chief operating officer, Mike Butcher, ...",Current Chief Operating Officer Mike Butcher a...
1,The world's two largest automakers said their ...,Domestic sales at both GM and No. 2 Ford Motor...
2,According to the federal Centers for Disease C...,The Centers for Disease Control and Prevention...
3,A tropical storm rapidly developed in the Gulf...,A tropical storm rapidly developed in the Gulf...
4,The company didn't detail the costs of the rep...,But company officials expect the costs of the ...
...,...,...
5172,Twice Sparrow sold the island twice to Thomas ...,Sparrow twice sold the island to Thomas Polloc...
5173,The name in Tupi means `` insensitive stone ''...,"The name in Tupi means '' hard stone `` , '' i..."
5174,"The company has branches in Tokyo , based in t...",The company has branches in Tokyo based in Sai...
5175,The modern coat of arms of Bavaria was designe...,The modern coat of arms of Bavaria was designe...


In [5]:
dataset_test = Dataset.from_pandas(df_test)
dataset_train = Dataset.from_pandas(df_train)

In [6]:
def preprocess_function(examples):
    prefix = 'paraphrase: '
    inputs = [prefix + doc for doc in examples["in"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["expected"], max_length=512, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_test = dataset_test.map(preprocess_function, batched=True)
tokenized_train = dataset_train.map(preprocess_function, batched=True)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/59 [00:00<?, ?ba/s]

In [16]:
tokenized_train_small = tokenized_train.shuffle(seed=42).select(range(5000))
tokenized_test_small = tokenized_test.shuffle(seed=42).select(range(500))

In [8]:
print(len(tokenized_train[0]['input_ids']))
print(len(tokenized_train[0]['attention_mask']))
print(len(tokenized_train[0]['labels']))

512
512
512


In [9]:
from transformers.adapters import T5AdapterModel, AutoAdapterModel, AutoModelWithHeads
from transformers import AdapterTrainer, TrainingArguments


In [10]:
model = AutoAdapterModel.from_pretrained(base_path)

Some weights of the model checkpoint at t5-small were not used when initializing T5AdapterModel: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5AdapterModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5AdapterModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of T5AdapterModel were not initialized from the model checkpoint at t5-small and are newly initialized: ['decoder.embed_tokens.weight', 'encoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
model.add_adapter(adapter_path)
model.add_seq2seq_lm_head(adapter_path)
model.train_adapter(adapter_path)

In [17]:
tokenized_dataset = {
    "train": tokenized_train_small.remove_columns(['in', 'expected']),
    "test": tokenized_test_small.remove_columns(['in', 'expected'])
}

training_args = TrainingArguments(
  output_dir="./checkpoints_adapter", 
  do_train=True,
  remove_unused_columns=False,
  learning_rate=1e-4,
  num_train_epochs=3,
  weight_decay=0.01,
)

trainer = AdapterTrainer(
  model=model,
  args=training_args,
  tokenizer=tokenizer,
  train_dataset=tokenized_dataset['train'],
  eval_dataset=tokenized_dataset['test']
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [18]:
tokenized_dataset

{'test': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 500
 }), 'train': Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 5000
 })}

In [19]:
# trainer.compute_loss()
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1875


Step,Training Loss
500,0.048700
1000,0.047800
1500,0.047100


Saving model checkpoint to ./checkpoints_adapter/checkpoint-500
Configuration saved in ./checkpoints_adapter/checkpoint-500/paraphrase/adapter_config.json
Module weights saved in ./checkpoints_adapter/checkpoint-500/paraphrase/pytorch_adapter.bin
Configuration saved in ./checkpoints_adapter/checkpoint-500/paraphrase/head_config.json
Module weights saved in ./checkpoints_adapter/checkpoint-500/paraphrase/pytorch_model_head.bin
Configuration saved in ./checkpoints_adapter/checkpoint-500/paraphrase/head_config.json
Module weights saved in ./checkpoints_adapter/checkpoint-500/paraphrase/pytorch_model_head.bin
Configuration saved in ./checkpoints_adapter/checkpoint-500/paraphrase/head_config.json
Module weights saved in ./checkpoints_adapter/checkpoint-500/paraphrase/pytorch_model_head.bin
tokenizer config file saved in ./checkpoints_adapter/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./checkpoints_adapter/checkpoint-500/special_tokens_map.json
Saving model checkpoint 

TrainOutput(global_step=1875, training_loss=0.04763319498697917, metrics={'train_runtime': 941.1152, 'train_samples_per_second': 15.939, 'train_steps_per_second': 1.992, 'total_flos': 2048547225600000.0, 'train_loss': 0.04763319498697917, 'epoch': 3.0})

In [20]:
model.save_adapter("adapter_paraphrase2", "paraphrase")

Configuration saved in adapter_paraphrase2/adapter_config.json
Module weights saved in adapter_paraphrase2/pytorch_adapter.bin
Configuration saved in adapter_paraphrase2/head_config.json
Module weights saved in adapter_paraphrase2/pytorch_model_head.bin


In [22]:
!zip -r 'adapter2.zip' 'adapter_paraphrase2'

  adding: adapter_paraphrase2/ (stored 0%)
  adding: adapter_paraphrase2/head_config.json (deflated 44%)
  adding: adapter_paraphrase2/pytorch_adapter.bin (deflated 8%)
  adding: adapter_paraphrase2/adapter_config.json (deflated 60%)
  adding: adapter_paraphrase2/pytorch_model_head.bin (deflated 53%)
